In [18]:
import pandas as pd
from math import isnan

In [19]:
GT_plates = pd.read_excel("PlatesLateral.ods", engine="odf")
results = pd.read_csv('../out/results.csv')
results_videos = pd.read_csv('../out/results_videos.csv')
GT_plates = GT_plates[GT_plates['Plate'].notna()]
GT_plates['Plate'] = GT_plates['Plate'].apply(lambda x: str(x).replace('“','').replace('”',''))
GT_plates

,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,Y
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,Y
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,N
134,w3,EHS,0718,NaN,NaN,Y
135,w3,Rare,SN-33XG,NaN,NaN,Y


In [20]:
results

,Model,Video,Frame,Pred_plate,Confidence
0,a109,a109e_takeoff,0,NaN,0.000000
1,a109,a109e_takeoff,1,NaN,0.000000
2,a109,a109e_takeoff,2,405,0.680593
3,a109,a109e_takeoff,3,NaN,0.000000
4,a109,a109e_takeoff,4,NaN,0.000000
...,...,...,...,...,...
4097,w3,Saving,102,NaN,0.000000
4098,w3,Saving,103,NaN,0.000000
4099,w3,Saving,104,NaN,0.000000
4100,w3,Saving,105,NaN,0.000000


In [21]:
results_videos

,Model,Video,Frame,Pred_plate,Confidence
0,a109,corporate,0,NaN,0.000000
1,a109,corporate,1,N2SM,0.895998
2,a109,corporate,2,NaN,0.000000
3,a109,corporate,3,N738AK,0.999948
4,a109,corporate,4,N93AK,0.931075
...,...,...,...,...,...
12286,w3,Rare,48,NaN,0.000000
12287,w3,Rare,49,NaN,0.000000
12288,w3,Rare,50,NaN,0.000000
12289,w3,Rare,51,NaN,0.000000


In [5]:
# Analyze results from images

In [22]:
n = GT_plates.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates.iterrows():
    #Get plates recognized in that video
    preds = results.loc[(results['Model'] == row['Model']) & (results['Video'] == row['Video'])]
    #If a frame interval is defined, get only plates in that intervals
    if not isnan(row['Init_frame']):
        preds = preds[preds['Frame'].isin(range(int(row['Init_frame']),int(row['End_frame']+1)))]
    #Check if plate was found
    #print(row['Plate'])
    #print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode(dropna=True)):
        democracy_plate = preds['Pred_plate'].mode(dropna=True)[0]
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates['Detected'] = detected
GT_plates.to_excel("PlatesLateral.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates

0.6058394160583942 0.5036496350364964


,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,Y
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,Y
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,Y
134,w3,EHS,0718,NaN,NaN,Y
135,w3,Rare,SN-33XG,NaN,NaN,Y


In [16]:
# Analyze results from videos

In [17]:
n = GT_plates.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates.iterrows():
    #Get plates recognized in that video
    preds = results_videos.loc[(results_videos['Model'] == row['Model']) & (results_videos['Video'] == row['Video'])]
    #Check if plate was found
    #print(row['Plate'])
    #print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode(dropna=True)):
        democracy_plate = preds['Pred_plate'].mode(dropna=True)[0]
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates['Detected'] = detected
GT_plates.to_excel("PlatesLateral.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates

0.5182481751824818 0.29927007299270075


,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,Y
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,Y
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,N
134,w3,EHS,0718,NaN,NaN,Y
135,w3,Rare,SN-33XG,NaN,NaN,Y
